In [3]:
import json
from pandas.io.json import json_normalize
exec(open('Einlesen.py').read())

loading ...
Finished
[2017, 7]


In [29]:
class getInfo():
    def __init__(self, data):
        self.data = data
        
    def mergeMatchGames(self):
        for x in range(9):
            match = self.data[x]
            print(self.getTeam1Name(match)) 
            print(self.getTeam2Name(match)) 
            print(self.getHalfResult(match)) 
            print(self.getEndResult(match)) 
            print(self.getGoalTimeTeam1(match)) 
            print(self.getGoalTimeTeam2(match)) 
            print(self.getSpectators(match)) 
            print(self.getDateTime(match)) 
            print(self.getPenaltyTeam1(match)) 
            print(self.getPenaltyTeam2(match)) 
            print(self.getScorerTeam1(match))
            print(self.getScorerTeam2(match))
            print('---') 
   
    def getTeam1Name(self,match):
        return match['Team1']['TeamName']
    
    def getTeam2Name(self,match):
        return match['Team2']['TeamName']
    
    def getHalfResult(self,match):
        return [match['MatchResults'][0]['PointsTeam1'], match['MatchResults'][0]['PointsTeam2']]

    def getEndResult(self,match):
        return [match['MatchResults'][1]['PointsTeam1'], match['MatchResults'][1]['PointsTeam2']]
    
    def mergeInfoTeam(self, match, Info, teamNumber):
        info = np.array([])
        team = np.array([])
        for x in match['Goals']:
            team = np.append(team, x['ScoreTeam'+str(teamNumber)])
            info = np.append(info, x[Info])
        if(len(team)==0):
            mask = None
        elif(len(team)>0):
            mask = np.append(team[0], team[1:]-team[:-1]).astype(bool)
        else:
            mask = team[0].astype(bool)
        return info[mask]
    
    def getGoalTimeTeam1(self, match):
        return self.mergeInfoTeam(match, 'MatchMinute', 1)
    
    def getGoalTimeTeam2(self, match):
        return self.mergeInfoTeam(match, 'MatchMinute', 2)
    
    def getPenaltyTeam1(self, match):
        return self.mergeInfoTeam(match, 'IsPenalty', 1).astype(bool)
    
    def getPenaltyTeam2(self, match):
        return self.mergeInfoTeam(match, 'IsPenalty', 2).astype(bool)
    
    def getScorerTeam1(self, match):
        return self.mergeInfoTeam(match, 'GoalGetterName', 1)
    
    def getScorerTeam2(self, match):
        return self.mergeInfoTeam(match, 'GoalGetterName', 2)
    
    def getSpectators(self,match):
        return match['NumberOfViewers']
    
    def getDateTime(self,match):
        return match['MatchDateTime']
    

In [30]:
class converter(downloader, getInfo):
    def __init__(self):
        self.status = None
        self.lastConvMatchday = None
    
    def saveLastConvMatchday(self):
        ''' Save the lastConvMatchday'''
        if(self.lastConvMatchday[1] <=1):
            lastConvMatchday = [self.lastConvMatchday[0]-1, 34]
        else:
            lastConvMatchday = [self.lastConvMatchday[0],self.lastConvMatchday[1]-1]
        with open('info.json', 'w') as file:
            file.write(json.dumps({'LastConvMatchday': lastConvMatchday}))
            
    def readLastConvMatchday(self):
        '''Read the last lastConvMatchday '''
        with open('info.json') as file:
            dataLoaded = json.load(file)
        return dataLoaded['LastConvMatchday']
    
    def updateSeason(self):
        actuellSeason = downloader().readLastLoadMatchday()[0]
        if(os.path.exists('season/'+str(actuellSeason)==False)):
            print(actuellSeason)
    
    def loadMatchday(self, year, day):
        season_day = (str(year) + '/' + str(day))
        with open(season_day+'.json', 'r') as f:#(str(year)+'/'+ str(day) + 
            return json.load(f)
    
    def testClass(self):
        data = self.loadMatchday(2017,1)
        getInfo(data).mergeMatchGames()

In [31]:
#converter().loadMatchday(2017,2)[4]

In [32]:
converter().testClass()

Bayern München
Bayer 04 Leverkusen
[2, 0]
[3, 1]
[  9.  19.  52.]
[ 66.]
75000
2017-08-18T20:30:00
[False False  True]
[False]
['Niklas Süle' 'Tolisso' 'Lewandowski, Robert']
[' Admir Mehmedi']
---
TSG 1899 Hoffenheim
Werder Bremen
[0, 0]
[1, 0]
[ 84.]
[]
30150
2017-08-19T15:30:00
[False]
[]
['Andrej Kramaric']
[]
---
Hertha BSC
VfB Stuttgart
[0, 0]
[2, 0]
[ 46.  62.]
[]
44751
2017-08-19T15:30:00
[False False]
[]
['Mathew Leckie' 'Mathew Leckie']
[]
---
Hamburger SV
FC Augsburg
[1, 0]
[1, 0]
[ 8.]
[]
49449
2017-08-19T15:30:00
[False]
[]
['N. Müller']
[]
---
1. FSV Mainz 05
Hannover 96
[0, 0]
[0, 1]
[]
[ 73.]
28279
2017-08-19T15:30:00
[]
[False]
[]
['Harnik']
---
VfL Wolfsburg
Borussia Dortmund
[0, 2]
[0, 3]
[]
[ 23.  27.  60.]
30000
2017-08-19T15:30:00
[]
[False False False]
[]
['Christian Pulisic' 'Barta' 'Pierre-Emerick Aubameyang']
---
FC Schalke 04
RB Leipzig
[1, 0]
[2, 0]
[ 44.  73.]
[]
61435
2017-08-19T18:30:00
[ True False]
[]
['Nabil Bentaleb' 'Konoplyanka']
[]
---
SC Freiburg
